In [16]:
from GroundProjection import *
from IntersectArea import *
from ColumnProjection import *
# from shapely.geometry import Polygon

from 求太阳各角度 import *
from 阳光向量 import *
from 求镜面顶点 import *
from 求镜面法向量 import *

from judge import *  # 读取距离矩阵并分析

import numpy as np
import pandas as pd


In [13]:
"函数测试"
print(GroundProjection(np.array([1,2,3]),np.array([-1,-3,-4])))
print(IntersectArea([[1,1],[2,1],[2,3],[1,3]],[[1,2],[1.5,1],[3,4],[2,4]]))
vector = np.array([200,0,76])
points = np.array([[-197.4,3,2.5],[-197.4,-3,2.5],[-202.60,-3,5.5],[-202.60,3,5.5]])
print(ColumnProjection(vector,points)[:,[0,2]])

a
[[   0.       3.      77.512]
 [-197.4     -3.       2.5  ]
 [-202.6     -3.       5.5  ]
 [-202.6      3.       5.5  ]]
[[   0.       3.      77.512]
 [   0.      -3.      77.512]
 [-202.6     -3.       5.5  ]
 [-202.6      3.       5.5  ]]
[[ 0.00000000e+00  3.00000000e+00  7.75120000e+01]
 [ 0.00000000e+00 -3.00000000e+00  7.75120000e+01]
 [-2.84217094e-14 -3.00000000e+00  8.24880000e+01]
 [-2.02600000e+02  3.00000000e+00  5.50000000e+00]]
[[ 0.00000000e+00  3.00000000e+00  7.75120000e+01]
 [ 0.00000000e+00 -3.00000000e+00  7.75120000e+01]
 [-2.84217094e-14 -3.00000000e+00  8.24880000e+01]
 [-2.84217094e-14  3.00000000e+00  8.24880000e+01]]
[[ 3.    77.512]
 [-3.    77.512]
 [-3.    82.488]
 [ 3.    82.488]]


In [19]:
"距离矩阵"

#读取数据
data = pd.read_excel('附件.xlsx')
data = np.array(data)

# 计算每个样本的欧式
def distance(P1, P2):
    return np.sqrt(np.sum((P1 - P2) ** 2))

# 创建result矩阵，用于存放每个样本之间的距离
result = np.zeros((len(data), len(data)))
for i in range(len(data)):
    for j in range(len(data)):
        if i < j:
            result[i][j] = distance(data[i], data[j])
        else:
            result[i][j] = 0
# 把上三角矩阵result转化为对称矩阵
for i in range(len(data)):
    for j in range(len(data)):
        if i > j:
            result[i][j] = result[j][i]

In [20]:
data

array([[107.25 ,  11.664],
       [105.36 ,  23.191],
       [102.235,  34.447],
       ...,
       [334.636, -40.95 ],
       [336.233, -24.609],
       [337.032,  -8.21 ]])

In [14]:
"计算镜子阴影最大影响范围"
"""
1/21  21
2/21  52
3/21  80
4/21  111
5/21  141
6/21  172
7/21  202
8/21  233
9/21  263
10/21 294
11/21 325
12/21 355
"""

min_sun_angle = 1000

phi = 39.4/180*np.pi
for day in [21,52,80,111,141,172,202,233,263,294,325,355]:
    day-=80
    for hour in [9,10.5,12,13.5,15]:
        __temp_min_sun_angle = get_alpha_s(phi,day,hour)
        if __temp_min_sun_angle < min_sun_angle:
            min_sun_angle = __temp_min_sun_angle
            print(min_sun_angle/np.pi*180,day,hour)

print(min_sun_angle/np.pi*180)

tall = 7

max_range = 7/np.tan(min_sun_angle)

max_range

17.4309154601833 -59 9
16.829321995177068 245 9
14.404529570653825 275 9
14.404529570653825


27.254254940193462

In [ ]:
TC=np.array([0,0,80])
# 集热塔大小
TOWER = np.array([3.5,-4],[3.5,4],[-3.5,4],[-3.5,-4])

def e_mirror_rate(ID,phi,day,hour,):
    '''
        
        返回 单面镜子 集热塔截断面积占比  截断效率

        ID:镜子编号
        维度phi 时间day hour  -- 太阳照射角度
        镜子中心坐标，  -- 镜子法向量&四角坐标
    '''
    mirror_center_point = data[ID]
    # 计算镜子投影位置并旋转
    sun_alpha = get_alpha_s(phi,day,hour)
    sun_vector = get_sun_vector(sun_alpha,get_gamma_s(phi,day,hour))
    mirror_column = get_mirror_out_vector(mirror_center_point)
    mirror_points = get_mirror_point(mirror_center_point,get_mirror_normal_vector(mirror_center_point,sun_vector,mirror_column))



    "截断效率"
    x_points = ColumnProjection(mirror_column,mirror_points,mirror_center_point)
    # 坐标二维化
    points = x_points[:,[0,2]]

    # 计算截断面积
    cut_area,area = IntersectArea(tower,points)
    # 计算投射总面积
    cut_rate = area/cut_area

    
    "阴影遮挡效率"
    #读取附近镜子
    max_range = 7/np.tan(sun_alpha)

    nearby = judge(result,ID,max_range)
    

